In [1]:
%pip install requests pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import itertools
from datetime import datetime, timedelta, date

import requests
import pandas as pd

In [38]:
token_env = 'test'

AMADEUS_CLIENT_ID = 'o3NmGGegeKqA4pS5IXXyuu9YEb5iGsD3'
AMADEUS_CLIENT_SECRET = '57oIRKGwGNNRHm6C'

# Token- und Offers-Endpoints
if token_env == 'test':
    token_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
    offers_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
else:
    token_url = 'https://api.amadeus.com/v1/security/oauth2/token'
    offers_url = 'https://api.amadeus.com/v2/shopping/flight-offers'

print(f"Verwende {token_env}-Endpoint für Token: {token_url}")

# Zeitraum: 1. Januar 2025 bis heute
dt_start = date(2025, 1, 1)
dt_end = datetime.utcnow().date()
dates = [(dt_start + timedelta(days=i)).isoformat() for i in range((dt_end - dt_start).days + 1)]

# IATA-Codes für Abfragen
iata_codes = [
    "IST", "BER", "LHR", "CDG", "FCO", "MAD", "KBP", "WAW",
    "OTP", "AMS", "ARN", "BRU", "ATH", "PRG", "LIS", "BUD",
    "BEG", "VIE", "ZRH", "SOF", "CPH", "HEL", "OSL", "DUB",
]

print(f"Abfragen für {len(dates)} Tage von {dt_start} bis {dt_end} für {len(iata_codes)} Städte.")

Verwende test-Endpoint für Token: https://test.api.amadeus.com/v1/security/oauth2/token
Abfragen für 159 Tage von 2025-01-01 bis 2025-06-08 für 24 Städte.


In [39]:
# OAuth2-Token von Amadeus abrufen
def get_access_token(client_id: str, client_secret: str) -> str:
    data = {'grant_type': 'client_credentials'}
    if AMADEUS_ENV == 'test':
        # Sandbox: Form-Params
        resp = requests.post(token_url, data={**data, 'client_id': client_id, 'client_secret': client_secret},
                             headers={'Content-Type': 'application/x-www-form-urlencoded'})
    else:
        # Produktion: Basic Auth
        resp = requests.post(token_url, data=data, auth=(client_id, client_secret))
    if resp.status_code != 200:
        try:
            err = resp.json()
            desc = err.get('error_description') or err.get('description') or resp.text
        except ValueError:
            desc = resp.text
        raise RuntimeError(f"Token-Fehler ({resp.status_code}): {desc}")
    token_json = resp.json()
    access_token = token_json.get('access_token')
    if not access_token:
        raise RuntimeError(f"Kein access_token erhalten: {token_json}")
    return access_token


In [40]:
def get_flight_prices(token: str, origin: str, destination: str, departure_date: str) -> dict:
    url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
    headers = {'Authorization': f'Bearer {token}'}
    params = {
        'originLocationCode': origin,
        'destinationLocationCode': destination,
        'departureDate': departure_date,
        'adults': 1,
        'currencyCode': 'EUR',
        'max': 250
    }
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()

In [42]:
def parse_response(data: dict) -> list:
    records = []
    for offer in data.get('data', []):
        price_info = offer.get('price', {})
        itineraries = offer.get('itineraries', [])
        if not itineraries:
            continue
        segments = itineraries[0].get('segments', [])
        if not segments:
            continue
        rec = {
            'FetchedAt': datetime.utcnow().isoformat(),
            'Origin': segments[0]['departure']['iataCode'],
            'Destination': segments[-1]['arrival']['iataCode'],
            'DepartureDate': segments[0]['departure']['at'],
            'MinPrice': price_info.get('total'),
            'Currency': price_info.get('currency'),
            'NumberOfSegments': len(segments)
        }
        records.append(rec)
    return records

In [43]:
token = get_access_token(AMADEUS_CLIENT_ID, AMADEUS_CLIENT_SECRET)
city_data = {orig: [] for orig in iata_codes}

for origin, destination in itertools.permutations(iata_codes, 2):
    for d in dates:
        try:
            data = get_flight_prices(token, origin, destination, d)
            recs = parse_response(data)
            if recs:
                city_data[origin].extend(recs)
        except Exception:
            continue
    print(f"{origin}: {len(city_data[origin])} Datensätze gesammelt")

IST: 13 Datensätze gesammelt
IST: 24 Datensätze gesammelt
IST: 47 Datensätze gesammelt
IST: 61 Datensätze gesammelt
IST: 76 Datensätze gesammelt
IST: 76 Datensätze gesammelt
IST: 85 Datensätze gesammelt
IST: 97 Datensätze gesammelt
IST: 109 Datensätze gesammelt
IST: 117 Datensätze gesammelt
IST: 126 Datensätze gesammelt
IST: 136 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt
IST: 147 Datensätze gesammelt


KeyboardInterrupt: 

In [44]:
os.makedirs('PricesCSV', exist_ok=True)
for origin, recs in city_data.items():
    if not recs:
        print(f"Keine Daten für {origin}")
        continue
    df = pd.DataFrame(recs)
    path = os.path.join('PricesCSV', f"amadeus_prices_{origin}.csv")
    df.to_csv(path, index=False)
    print(f"Gespeichert: {path} ({len(df)} Einträge)")


Gespeichert: PricesCSV/amadeus_prices_IST.csv (147 Einträge)
Keine Daten für BER
Keine Daten für LHR
Keine Daten für CDG
Keine Daten für FCO
Keine Daten für MAD
Keine Daten für KBP
Keine Daten für WAW
Keine Daten für OTP
Keine Daten für AMS
Keine Daten für ARN
Keine Daten für BRU
Keine Daten für ATH
Keine Daten für PRG
Keine Daten für LIS
Keine Daten für BUD
Keine Daten für BEG
Keine Daten für VIE
Keine Daten für ZRH
Keine Daten für SOF
Keine Daten für CPH
Keine Daten für HEL
Keine Daten für OSL
Keine Daten für DUB


In [45]:
import glob
files = sorted(glob.glob('PricesCSV/amadeus_prices_*.csv'))
if files:
    display(pd.read_csv(files[0]).head())
else:
    print("Keine CSVs in PricesCSV gefunden.")

,FetchedAt,Origin,Destination,DepartureDate,MinPrice,Currency,NumberOfSegments
0,2025-06-08T17:14:19.235249,IST,BER,2025-06-08T20:50:00,175.21,EUR,2
1,2025-06-08T17:14:19.235266,IST,BER,2025-06-08T20:50:00,175.21,EUR,2
2,2025-06-08T17:14:19.235271,SAW,BER,2025-06-08T21:00:00,182.00,EUR,2
3,2025-06-08T17:14:19.235276,SAW,BER,2025-06-08T22:20:00,191.40,EUR,2
4,2025-06-08T17:14:19.235280,SAW,BER,2025-06-08T22:20:00,191.40,EUR,2
